In [4]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
random.seed(42)

In [5]:
df=pd.read_csv('AB_test_data.csv')

In [6]:
df.head()

,purchase_TF,Variant,date,id,Unnamed: 4,1.define hypothesis
0,False,A,2019/11/8,0x25b44a,NaN,H0: p0 = p1
1,False,B,2020/8/27,0x46271e,NaN,Ha: p0 \= p1
2,False,A,2020/6/11,0x80b8f1,NaN,a = 0.05
3,False,B,2020/8/22,0x8d736d,NaN,2. calculate z-score
4,False,A,2020/8/5,0x96c9c8,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   purchase_TF          130000 non-null  bool   
 1   Variant              130000 non-null  object 
 2   date                 130000 non-null  object 
 3   id                   130000 non-null  object 
 4   Unnamed: 4           0 non-null       float64
 5   1.define hypothesis  4 non-null       object 
dtypes: bool(1), float64(1), object(4)
memory usage: 5.1+ MB


In [8]:
P_a=float(df.query('Variant=="A"').query('purchase_TF==True')['id'].count())/float(df.query('Variant=="A"')['id'].count())

In [9]:
P_b=float(df.query('Variant=="B"').query('purchase_TF==True')['id'].count())/float(df.query('Variant=="B"')['id'].count())

In [10]:
P_a

0.149616

In [11]:
P_b

0.1766

In [12]:
from statsmodels.stats.proportion import proportions_ztest

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   purchase_TF          130000 non-null  bool   
 1   Variant              130000 non-null  object 
 2   date                 130000 non-null  object 
 3   id                   130000 non-null  object 
 4   Unnamed: 4           0 non-null       float64
 5   1.define hypothesis  4 non-null       object 
dtypes: bool(1), float64(1), object(4)
memory usage: 5.1+ MB


In [14]:
control_results = df[(df['purchase_TF'] == True) &(df['Variant']=='A')]['purchase_TF']
control_results

10        True
19        True
22        True
37        True
45        True
          ... 
129978    True
129980    True
129982    True
129988    True
129989    True
Name: purchase_TF, Length: 18702, dtype: bool

In [15]:
treatment_results = df[(df['purchase_TF'] == True)&(df['Variant']=='B')]['purchase_TF']
treatment_results

181       True
415       True
429       True
553       True
726       True
          ... 
128735    True
128881    True
129016    True
129048    True
129798    True
Name: purchase_TF, Length: 883, dtype: bool

In [16]:
n_treat = df[df['Variant']=='B']['id'].count()

In [17]:
n_treat

5000

In [18]:
n_con = df[df['Variant']=='A']['id'].count()
n_treat = df[df['Variant']=='B']['id'].count()
successes = [treatment_results.sum(),control_results.sum() ]
nobs = [n_treat, n_con]

In [19]:
n_con

125000

In [20]:
successes

[883, 18702]

In [21]:
z_stat, pval = proportions_ztest(successes, nobs=nobs)

In [22]:
z_stat

5.2304877183522045

In [24]:
#optimal_n=((1.96*np.sqrt(2*p_bar*(1-p_bar))+0.2*np.sqrt(P_b*(1-P_b)+P_a*(1-P_a)))**2)/((0.02)**2)

In [26]:
#optimal=optimal_n.round()

In [27]:
treatment=df[df['Variant']=='B']

In [28]:
control=df[df['Variant']=='A']

In [29]:
from random import sample

In [30]:
control1=control.sample(n=3184)

In [31]:
treatment1=treatment.sample(n=3184)

In [32]:
treatment_results1= treatment1[treatment1['purchase_TF'] == True]['purchase_TF']
control_results1= control1[control1['purchase_TF'] == True]['purchase_TF']

In [33]:
successes1 = [control_results1.sum(), treatment_results1.sum()]

In [34]:
z_stat1, pval1 = proportions_ztest(successes1, nobs=[3184,3184])

In [35]:
z_stat1

-3.030078544089735

In [36]:
pval1

0.0024449014784504357

In [37]:
P_a= control_results1.sum()/3184
P_a

0.14761306532663315

In [38]:
P_b= treatment_results1.sum()/3184
P_b

0.17556532663316582

In [39]:
random_num_dict = {}
for i in range(10):
    random_num_dict[i] = [control.sample(n=3184),treatment.sample(n=3184)]
#print(random_num_dict)

z_stats_list = []
p_val_list = []
P_a_list = []
P_b_list = []

for key in random_num_dict.keys():
    #obtaining the results(randomized control/treatment group) by key
    df_control_result = random_num_dict[key][0]
    control_results = df_control_result[control['purchase_TF'] == True]['purchase_TF']
    df_treatment_result = random_num_dict[key][1]
    treatment_results = df_treatment_result[treatment['purchase_TF'] == True]['purchase_TF']
    
    successes = [treatment_results.sum(),control_results.sum()]
    
    z_stat, p_val = proportions_ztest(successes, nobs=[3184,3184])
    z_stats_list.append(z_stat)
    p_val_list.append(p_val)
    
    P_a = control_results.sum()/3184
    P_b = treatment_results.sum()/3184
    
    P_a_list.append(P_a)
    P_b_list.append(P_b)
    
    
print(z_stats_list)
print(p_val_list)
print(P_a_list)
print(P_b_list)
#each list is indiced according to range(10), e.g. z_stats_1 = z_stats_list[0], p_val_1 = p_val_list[0].

[4.310340001635158, 1.9088154087269509, 4.574739832886086, 3.4279081902767774, 3.0742570696740414, 3.4660549170759873, 3.614997406465741, 1.9801112709506952, 3.0206319696498345, 3.1939257700472834]
[1.6300373075495345e-05, 0.05628590839900456, 4.768121252995879e-06, 0.0006082511245780345, 0.002110275078541195, 0.0005281553893490519, 0.0003003506284811257, 0.04769102695703615, 0.0025224775642887034, 0.0014035220123992608]
[0.14290201005025124, 0.15232412060301506, 0.13944723618090452, 0.14698492462311558, 0.1504396984924623, 0.14258793969849246, 0.1407035175879397, 0.1513819095477387, 0.14886934673366833, 0.1560929648241206]
[0.1827889447236181, 0.16991206030150754, 0.1815326633165829, 0.17870603015075376, 0.17902010050251257, 0.17430904522613067, 0.17368090452261306, 0.16959798994974876, 0.176821608040201, 0.18624371859296482]


C:\Users\Ruofan Mo\AppData\Local\Temp\ipykernel_37996\688659106.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control_results = df_control_result[control['purchase_TF'] == True]['purchase_TF']
C:\Users\Ruofan Mo\AppData\Local\Temp\ipykernel_37996\688659106.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  treatment_results = df_treatment_result[treatment['purchase_TF'] == True]['purchase_TF']


In [40]:
results = pd.DataFrame(list(zip(z_stats_list, P_a_list,P_b_list)),
               columns =['Z_score', 'P_a','P_b'])
results
    

,Z_score,P_a,P_b
0,4.310340,0.142902,0.182789
1,1.908815,0.152324,0.169912
2,4.574740,0.139447,0.181533
3,3.427908,0.146985,0.178706
4,3.074257,0.150440,0.179020
5,3.466055,0.142588,0.174309
6,3.614997,0.140704,0.173681
7,1.980111,0.151382,0.169598
8,3.020632,0.148869,0.176822
9,3.193926,0.156093,0.186244


In [41]:
!pip install sprt

In [52]:
import sprt as sprt
import numpy as np

test = sprt.SPRTNormal(alpha = 0.05, beta = 0.2, h0 = 0, h1 = 1, 
values = results['P_a'], 
variance = 1)

Decision:	Accept

    values  lower  upper
n                       
1    0.295 -1.058  3.273
2    0.435 -0.558  3.773
3    0.582 -0.058  4.273
4    0.732  0.442  4.773
5    0.875  0.942  5.273
6    1.015  1.442  5.773
7    1.167  1.942  6.273
8    1.316  2.442  6.773
9    1.472  2.942  7.273
10     NaN  3.442  7.773


In [46]:
test.plot

<bound method SPRT.plot of <sprt.sprt.SPRTNormal object at 0x0000021D27233310>>

In [49]:
import sprt as sprtb
import numpy as np

test = sprt.SPRTNormal(alpha = 0.05, beta = 0.2, h0 = 0, h1 = 1, 
values = results['P_b'], 
variance = 1)

Decision:	Accept

    values  lower  upper
n                       
1    0.353 -1.058  3.273
2    0.534 -0.558  3.773
3    0.713 -0.058  4.273
4    0.892  0.442  4.773
5    1.066  0.942  5.273
6    1.240  1.442  5.773
7    1.410  1.942  6.273
8    1.586  2.442  6.773
9    1.773  2.942  7.273
10     NaN  3.442  7.773


In [50]:
test.plot

<bound method SPRT.plot of <sprt.sprt.SPRTNormal object at 0x0000021D271FF910>>

In [ ]:
Z_stat=[]
pval=[]
treatment_results = []
control_results=[]
successes=[]
nobs=[optimal,optimal]
for [i,j] in np.arange(10):

    control_sample=control.sample(n=int(optimal))
    treatment_sample=treatment.sample(n=int(optimal))
    treatment_results[i] = treatment_sample[treatment_sample['purchase_TF'] == True]['purchase_TF']
    control_results[i] = control_sample[control_sample['purchase_TF'] == True]['purchase_TF']
    successes[i] = [control_results[i].sum(), treatment_results[i].sum()]
    
    Z_stat[i], pval[i] = proportions_ztest(successes[i], nobs=nobs)
    
    print (Z_stat)
    
    


In [ ]:
for i in np.arange(10):
    print(i)
for j in np.arange(10):
    print(j)

